In [1]:
using Dates
using PRMaps
using Plots
using Healpix

using thesis

import Pandas as Pd
import Stripeline as Sl

In [2]:
t_start = DateTime(2022, 1, 1, 12, 0, 0)

sky_model = "c1s0d0"
nside = 512
obs_days = 365
 
setup = PRMaps.Setup(
    sampling_freq_Hz = 50.,
    total_time_s = 24. * 3600. * obs_days
)

camera = Sl.CameraAngles()

tel_ang = Sl.TelescopeAngles(
    wheel2ang_0_rad = deg2rad(1.0/60.0)
)

MethodError: MethodError: no method matching Stripeline.TelescopeAngles(; wheelang2_0_rad=0.0002908882086657216)
Closest candidates are:
  Stripeline.TelescopeAngles(; wheel1ang_0_rad, wheel2ang_0_rad, wheel3ang_0_rad, forkang_rad, ωVAXang_rad, zVAXang_rad) at util.jl:493 got unsupported keyword argument "wheelang2_0_rad"
  Stripeline.TelescopeAngles(!Matched::Float64, !Matched::Float64, !Matched::Float64, !Matched::Float64, !Matched::Float64, !Matched::Float64) at ~/.julia/packages/Stripeline/JpMBp/src/scanning.jl:126 got unsupported keyword argument "wheelang2_0_rad"
  Stripeline.TelescopeAngles(!Matched::Any, !Matched::Any, !Matched::Any, !Matched::Any, !Matched::Any, !Matched::Any) at ~/.julia/packages/Stripeline/JpMBp/src/scanning.jl:126 got unsupported keyword argument "wheelang2_0_rad"

In [3]:
strip = Pd.query(Pd.read_pickle("../instruments/lspe_strip_instrument.pkl"), :(frequency==43.0))
signal = get_foreground_maps(strip, sky_model, nside)[1]
nothing

In [ ]:
ideal, pixel = makeErroredMapIQU(camera, tel_ang, signal, setup, t_start)

In [ ]:
plot(ideal.q)

In [ ]:
prova = Healpix.PolarizedHealpixMap{Float64, RingOrder}(512)
prova = ideal
Healpix.saveToFITS(prova, "maps_365days/altitude.fits", typechar="D", extname="MAP")

In [ ]:
a = readPolarizedMapFromFITS("maps_365days/altitude.fits", 1, Float64)
plot(a.i)